In [1]:
from cortex import CortexDataset

[2019-12-05 11:11:37,268] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.


## Load Cortex dataset

In [2]:
cortex_dataset = CortexDataset(save_path="./data/", total_genes=558)

[2019-12-05 11:11:43,979] INFO - scvi.dataset.dataset | File /u/davidm/Documents/Cours/IFT6269/project/umontreal-ift6269-pgm/data/expression.bin already downloaded
[2019-12-05 11:11:50,949] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-12-05 11:11:50,950] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


#### No batch annotation :

In [3]:
np.unique(cortex_dataset.batch_indices)

array([0], dtype=uint16)

#### 7 neuronal cells lines in Cortex :

In [4]:
np.unique(cortex_dataset.labels)

array([0, 1, 2, 3, 4, 5, 6], dtype=uint16)